In [3]:
from pathlib import Path
import zipfile
import pandas as pd
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sb
from sklearn.model_selection import train_test_split
sklearn.set_config(display="diagram")
np.random.seed(42)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


import pandas as pd

# Replace the file ID with your own
def load_data():
  return pd.read_csv(Path("/content/drive/MyDrive/dataset.csv"))

#Load the data set
data = load_data()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
train.head()

ID  travel_fee             departure_time  departure_long  departure_lat  arrival_long  arrival_lat  occupancy
625101   2013-06-27 18:07:00.000000180         4.5  2013-06-27 18:07:00+00:00      -73.980940      40.759717    -73.976447    40.764835          2
1336914   2013-07-27 08:55:00.00000037        16.5  2013-07-27 08:55:00+00:00      -73.973800      40.750710    -74.014455    40.703702          1
2088427  2013-08-16 11:20:00.000000124         8.0  2013-08-16 11:20:00+00:00      -74.001810      40.740705    -73.987485    40.752102          1
1612856    2013-04-23 22:42:30.0000003        14.0  2013-04-23 22:42:30+00:00      -73.993854      40.720611    -73.992844    40.760073          1
1847080  2013-04-20 19:34:00.000000256         7.5  2013-04-20 19:34:00+00:00      -73.960283      40.690125    -73.987232    40.692187          1

In [5]:
train=train.loc[train['travel_fee']>0]
train=train.loc[train['occupancy']>0]
train.shape

(2077099, 8)

In [6]:
train = train.loc[train['departure_long']!=0]
train = train.loc[train['departure_lat']!=0]
train = train.loc[train['arrival_long']!=0]
train = train.loc[train['arrival_lat']!=0]

train.eq(0).sum()

ID                0
travel_fee        0
departure_time    0
departure_long    0
departure_lat     0
arrival_long      0
arrival_lat       0
occupancy         0
dtype: int64

In [7]:
num_attr = [
    'travel_fee', 'departure_long', 'departure_lat', 'arrival_long','arrival_lat','occupancy'
]

for column in num_attr:
    print("\nStatistics for", column)
    print("Minimum:", train[column].min())
    print("Maximum:", train[column].max())
    print("Mean:", train[column].mean())
    print("Standard deviation:", train[column].std())


Statistics for travel_fee
Minimum: 0.01
Maximum: 500.0
Mean: 12.564952158921718
Standard deviation: 10.662037892843736

Statistics for departure_long
Minimum: -740.021667
Maximum: 83.433358
Mean: -73.6110324757508
Standard deviation: 6.513531464487828

Statistics for departure_lat
Minimum: -180.0
Maximum: 79.849171
Mean: 40.38591856263423
Standard deviation: 6.4575019076644065

Statistics for arrival_long
Minimum: -1216.416667
Maximum: 73.937848
Mean: -73.61012416969521
Standard deviation: 6.529346133550137

Statistics for arrival_lat
Minimum: -180.0
Maximum: 73.995127
Mean: 40.386426162148815
Standard deviation: 6.457317453309253

Statistics for occupancy
Minimum: 1
Maximum: 9
Mean: 1.711048311260873
Standard deviation: 1.3767518400513625


In [8]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Calculate and add Haversine distance column to the DataFrame
train['distance_km'] = haversine_distance(train['departure_lat'],
                                                     train['departure_long'],
                                                     train['arrival_lat'],
                                                     train['arrival_long'])

# use distance instead of longitudes & latitudes

In [9]:
train['distance_km'].describe()

count    2.041760e+06
mean     3.626926e+00
std      3.419214e+01
min      0.000000e+00
25%      1.276033e+00
50%      2.192538e+00
75%      3.989618e+00
max      1.016583e+04
Name: distance_km, dtype: float64

In [10]:
train = train.loc[train['distance_km']<=100]
train.describe()

travel_fee  departure_long  departure_lat  arrival_long   arrival_lat     occupancy   distance_km
count  2.041558e+06    2.041558e+06   2.041558e+06  2.041558e+06  2.041558e+06  2.041558e+06  2.041558e+06
mean   1.256468e+01   -7.361071e+01   4.038595e+01 -7.361007e+01  4.038638e+01  1.711039e+00  3.396691e+00
std    1.065723e+01    6.457775e+00   6.457204e+00  6.457792e+00  6.457228e+00  1.376743e+00  3.704495e+00
min    1.000000e-02   -1.800000e+02  -1.800000e+02 -1.800000e+02 -1.800000e+02  1.000000e+00  0.000000e+00
25%    6.500000e+00   -7.399236e+01   4.073568e+01 -7.399164e+01  4.073478e+01  1.000000e+00  1.275943e+00
50%    9.500000e+00   -7.398208e+01   4.075286e+01 -7.398063e+01  4.075339e+01  1.000000e+00  2.192345e+00
75%    1.400000e+01   -7.396787e+01   4.076737e+01 -7.396498e+01  4.076811e+01  2.000000e+00  3.988797e+00
max    5.000000e+02    7.393785e+01   4.878085e+01  7.393785e+01  4.878085e+01  9.000000e+00  9.912915e+01

In [11]:
train = train.loc[train['travel_fee']<=300]

In [12]:
from datetime import datetime
train['departure_time'] = pd.to_datetime(train['departure_time']).dt.dayofweek + 1

train[["departure_time"]].describe()

departure_time
count    2.041554e+06
mean     4.038141e+00
std      1.956506e+00
min      1.000000e+00
25%      2.000000e+00
50%      4.000000e+00
75%      6.000000e+00
max      7.000000e+00

In [13]:
dayofweek_mapping = {
    1:'Monday',
    2:'Tuesday',
    3:'Wednesday',
    4:'Thursday',
    5:'Friday',
    6:'Saturday',
    7:'Sunday'
}

train['departure_day'] = train['departure_time'].replace(dayofweek_mapping)
train.head()

ID  travel_fee  departure_time  departure_long  departure_lat  arrival_long  arrival_lat  occupancy  distance_km departure_day
625101   2013-06-27 18:07:00.000000180         4.5               4      -73.980940      40.759717    -73.976447    40.764835          2     0.683420      Thursday
1336914   2013-07-27 08:55:00.00000037        16.5               6      -73.973800      40.750710    -74.014455    40.703702          1     6.249678      Saturday
2088427  2013-08-16 11:20:00.000000124         8.0               5      -74.001810      40.740705    -73.987485    40.752102          1     1.749944        Friday
1612856    2013-04-23 22:42:30.0000003        14.0               2      -73.993854      40.720611    -73.992844    40.760073          1     4.388799       Tuesday
1847080  2013-04-20 19:34:00.000000256         7.5               6      -73.960283      40.690125    -73.987232    40.692187          1     2.283660      Saturday

In [14]:
taxi = train.drop('travel_fee',axis=1)
taxi_fee = train['travel_fee'].copy()

#Split Dataset for ANN Model & Perform Preprocessing

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Split the dataset into train and test sets
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Define the input features (X_train) and the target variable (y_train)
X_train = train.drop(columns=['ID', 'travel_fee'])  # Drop nonfeat columns
y_train = train['travel_fee']  # Target variable

# Display the shape of X_train and y_train to confirm
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

X_train.info()

Shape of X_train: (2077246, 6)
Shape of y_train: (2077246,)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2077246 entries, 625101 to 2219110
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   departure_time  object 
 1   departure_long  float64
 2   departure_lat   float64
 3   arrival_long    float64
 4   arrival_lat     float64
 5   occupancy       int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 110.9+ MB


In [16]:
class HaversineDistanceTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        distances = haversine_distance(X['departure_lat'], X['departure_long'],
                                       X['arrival_lat'], X['arrival_long'])
        return np.c_[X, distances]

num_attrs = ['occupancy','departure_long','departure_lat', 'arrival_long','arrival_lat']
cat_attrs = ['departure_time']

num_pipeline = make_pipeline(
    SimpleImputer(strategy='mean'),  # Impute missing values
    #FunctionTransformer(remove_zero_rows, kw_args={'columns': num_attrs}),
    StandardScaler()
)

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore", sparse=False)  # Convert to dense array
    )

distance_pipeline = make_pipeline(
    HaversineDistanceTransformer()
)

preprocessing = ColumnTransformer([
    ('num', num_pipeline, num_attrs),
    #('cat', cat_pipeline, cat_attrs),
    ('distance', distance_pipeline, [])    # distance(km)
])

preprocessing

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['occupancy', 'departure_long',
                                  'departure_lat', 'arrival_long',
                                  'arrival_lat']),
                                ('distance',
                                 Pipeline(steps=[('haversinedistancetransformer',
                                                  HaversineDistanceTransformer())]),
                                 [])])

In [18]:
X_train_preprocessed = preprocessing.fit_transform(X_train)

X_train_preprocessed

array([[ 0.21130726, -0.13694576,  0.10849655, -0.13830497,  0.1223698 ],
       [-0.51572405, -0.13631108,  0.10753459, -0.14164298,  0.11517353],
       [-0.51572405, -0.13880093,  0.10646604, -0.13927437,  0.12087093],
       ...,
       [-0.51572405, -0.13883408,  0.10581456, -0.14139523,  0.11811499],
       [-0.51572405, -0.13624948,  0.10757624, -0.13955075,  0.11887072],
       [-0.51572405, -0.13505771,  0.110019  , -0.13781518,  0.12185009]])

#Build and Train Model

In [19]:
# X_train_preprocessed = preprocessing.fit_transform(X_train)


from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def build_model(input_shape):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='linear'))  # use linear regression for numerical output price
    return model

# Build and train the model
model = build_model(X_train_preprocessed.shape[1:])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_preprocessed, y_train, epochs=10, batch_size=64)

Epoch 1/10
32457/32457 [==============================] - 109s 3ms/step - loss: 110.9153
Epoch 2/10
32457/32457 [==============================] - 114s 4ms/step - loss: 75.4542
Epoch 3/10
32457/32457 [==============================] - 103s 3ms/step - loss: 69.5272
Epoch 4/10
32457/32457 [==============================] - 102s 3ms/step - loss: 62.0887
Epoch 5/10
32457/32457 [==============================] - 102s 3ms/step - loss: 46.5992
Epoch 6/10
32457/32457 [==============================] - 102s 3ms/step - loss: 39.6512
Epoch 7/10
32457/32457 [==============================] - 105s 3ms/step - loss: 31.9637
Epoch 8/10
32457/32457 [==============================] - 102s 3ms/step - loss: 28.8871
Epoch 9/10
32457/32457 [==============================] - 104s 3ms/step - loss: 28.0731
Epoch 10/10
32457/32457 [==============================] - 102s 3ms/step - loss: 27.6212


#Perform Hyperparameter Tuning
- Bruite force manual tuning? Using SciKit Wrapper?

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV


ModuleNotFoundError: No module named 'keras.wrappers'

In [ ]:
import tensorflow as tf
# Change number of neurons in each layer
tf.random.set_seed(42)

def build_model(input_shape):
    model = Sequential()
    model.add(tf.keras.layers.Dense(300, activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    return model

X_train_preprocessed = preprocessing.fit_transform(X_train)

model = build_model(X_train_preprocessed.shape[1:])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train_preprocessed, y_train, epochs=10, batch_size=64)

Epoch 1/10
32457/32457 [==============================] - 51s 2ms/step - loss: 105.4673 - mae: 6.4277
Epoch 2/10
32457/32457 [==============================] - 46s 1ms/step - loss: 71.9391 - mae: 5.2935
Epoch 3/10
32457/32457 [==============================] - 44s 1ms/step - loss: 69.0141 - mae: 5.1476
Epoch 4/10
32457/32457 [==============================] - 44s 1ms/step - loss: 58.5031 - mae: 4.7022
Epoch 5/10
32457/32457 [==============================] - 45s 1ms/step - loss: 38.9543 - mae: 3.6123
Epoch 6/10
32457/32457 [==============================] - 41s 1ms/step - loss: 29.6611 - mae: 2.9996
Epoch 7/10
32457/32457 [==============================] - 43s 1ms/step - loss: 28.3371 - mae: 2.8770
Epoch 8/10
32457/32457 [==============================] - 43s 1ms/step - loss: 27.8285 - mae: 2.8300
Epoch 9/10
32457/32457 [==============================] - 45s 1ms/step - loss: 27.5419 - mae: 2.7951
Epoch 10/10
32457/32457 [==============================] - 41s 1ms/step - loss: 27.3710 - 

In [ ]:
# Change optimiser, try Root Mean Square Propagation (RMSProp)

def build_model(input_shape):
    model = Sequential()
    model.add(tf.keras.layers.Dense(300, activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    return model


X_train_preprocessed = preprocessing.fit_transform(X_train)


model = build_model(X_train_preprocessed.shape[1:])
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train_preprocessed, y_train, epochs=10, batch_size=64)

# RMSProp has higher error than Adaptive Moment Estimation(adam)

Epoch 1/10
32457/32457 [==============================] - 43s 1ms/step - loss: 153.5132 - mae: 6.3187
Epoch 2/10
32457/32457 [==============================] - 43s 1ms/step - loss: 5755.5957 - mae: 5.5009
Epoch 3/10
32457/32457 [==============================] - 41s 1ms/step - loss: 60966.4844 - mae: 5.2970
Epoch 4/10
32457/32457 [==============================] - 41s 1ms/step - loss: 124897.6875 - mae: 5.1357
Epoch 5/10
32457/32457 [==============================] - 41s 1ms/step - loss: 119304.3750 - mae: 5.0801
Epoch 6/10
32457/32457 [==============================] - 41s 1ms/step - loss: 129765.7891 - mae: 5.0939
Epoch 7/10
32457/32457 [==============================] - 42s 1ms/step - loss: 131736.0156 - mae: 4.9181
Epoch 8/10
32457/32457 [==============================] - 42s 1ms/step - loss: 129467.7891 - mae: 4.8181
Epoch 9/10
32457/32457 [==============================] - 43s 1ms/step - loss: 137620.4531 - mae: 4.8511
Epoch 10/10
32457/32457 [==============================] - 43

In [ ]:
# Change optimiser, try Adaptive Gradient Algorithm (adagrad)

def build_model(input_shape):
    model = Sequential()
    model.add(tf.keras.layers.Dense(300, activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    return model


X_train_preprocessed = preprocessing.fit_transform(X_train)


model = build_model(X_train_preprocessed.shape[1:])
model.compile(optimizer='adagrad', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train_preprocessed, y_train, epochs=10, batch_size=64)

# AdaProp has higher error than Adaptive Moment Estimation(adam)

Epoch 1/10
32457/32457 [==============================] - 43s 1ms/step - loss: 123.9128 - mae: 6.8228
Epoch 2/10
32457/32457 [==============================] - 41s 1ms/step - loss: 121.1929 - mae: 6.7808
Epoch 3/10
32457/32457 [==============================] - 41s 1ms/step - loss: 120.7095 - mae: 6.7777
Epoch 4/10
32457/32457 [==============================] - 41s 1ms/step - loss: 120.3911 - mae: 6.7801
Epoch 5/10
32457/32457 [==============================] - 48s 1ms/step - loss: 120.1565 - mae: 6.7792
Epoch 6/10
32457/32457 [==============================] - 44s 1ms/step - loss: 119.9658 - mae: 6.7811
Epoch 7/10
32457/32457 [==============================] - 44s 1ms/step - loss: 119.8068 - mae: 6.7763
Epoch 8/10
32457/32457 [==============================] - 41s 1ms/step - loss: 119.6694 - mae: 6.7790
Epoch 9/10
32457/32457 [==============================] - 41s 1ms/step - loss: 119.5440 - mae: 6.7823
Epoch 10/10
32457/32457 [==============================] - 40s 1ms/step - loss: 11

In [21]:
# Choose Adaptive Moment Estimation as optimiser
# Increase Epochs from 10 to 50
import tensorflow as tf
def build_model(input_shape):
    model = Sequential()
    model.add(tf.keras.layers.Dense(300, activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    return model


X_train_preprocessed = preprocessing.fit_transform(X_train)


model = build_model(X_train_preprocessed.shape[1:])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train_preprocessed, y_train, epochs=50, batch_size=64)

# Final loss=22.9708, MAE=2.3981 --> much lower than Epochs=10

Epoch 1/50
32457/32457 [==============================] - 111s 3ms/step - loss: 101.5187 - mae: 6.3149
Epoch 2/50
32457/32457 [==============================] - 109s 3ms/step - loss: 71.9504 - mae: 5.2871
Epoch 3/50
32457/32457 [==============================] - 109s 3ms/step - loss: 71.8337 - mae: 5.1951
Epoch 4/50
32457/32457 [==============================] - 110s 3ms/step - loss: 66.2724 - mae: 5.0162
Epoch 5/50
32457/32457 [==============================] - 110s 3ms/step - loss: 57.4785 - mae: 4.4687
Epoch 6/50
32457/32457 [==============================] - 110s 3ms/step - loss: 40.5718 - mae: 3.6602
Epoch 7/50
32457/32457 [==============================] - 110s 3ms/step - loss: 31.8076 - mae: 3.1222
Epoch 8/50
32457/32457 [==============================] - 111s 3ms/step - loss: 29.4926 - mae: 2.9606
Epoch 9/50
32457/32457 [==============================] - 108s 3ms/step - loss: 28.2093 - mae: 2.8600
Epoch 10/50
32457/32457 [==============================] - 107s 3ms/step - loss: 

#Evaluate final model

In [22]:
X_test = test.drop(columns=['ID', 'travel_fee'])
y_test = test['travel_fee']


In [23]:
X_test

departure_time  departure_long  departure_lat  arrival_long  arrival_lat  occupancy
2286003  2013-01-05 01:51:00+00:00      -73.994628      40.761117    -73.965712    40.800445          1
1129435  2013-06-26 21:54:40+00:00      -73.960420      40.761856    -73.992941    40.760935          1
1278087  2013-12-07 22:09:00+00:00      -73.991902      40.737357    -73.999442    40.721807          1
143230   2013-01-26 18:28:25+00:00      -73.992588      40.754879    -73.992588    40.754879          1
938366   2013-05-20 14:07:04+00:00      -73.961901      40.759660    -73.952838    40.780360          2
...                            ...             ...            ...           ...          ...        ...
1928606  2013-05-30 09:39:00+00:00      -74.015718      40.711285    -74.010215    40.720742          1
2437758  2013-07-31 23:25:45+00:00      -74.002082      40.739658    -73.981274    40.729270          1
388546   2013-11-27 09:28:53+00:00      -73.977723      40.753422    -73.982586    40.760029          1
1752789  2013-01-24 18:47:44+00:00      -73.863641      40.769801    -73.963605    40.798043          1
913634   2013-05-07 14:47:13+00:00      -73.946910      40.771703    -73.951165    40.782732          1

[519312 rows x 6 columns]

In [24]:
y_test

2286003    10.5
1129435    11.0
1278087     7.0
143230      3.5
938366      9.5
           ... 
1928606     6.0
2437758     9.0
388546      4.5
1752789    32.8
913634      7.5
Name: travel_fee, Length: 519312, dtype: float64

In [25]:
X_test_preprocessed = preprocessing.fit_transform(X_test)

In [26]:
model.evaluate(X_test_preprocessed, y_test)

16229/16229 [==============================] - 37s 2ms/step - loss: 326.1104 - mae: 16.7734


[326.1103820800781, 16.773448944091797]

#Final thoughts
- Training loss (22.9708) and training metrics MAE (2.3981) are considerably low, which means that the model is fitting and training well on the data with the low errors.
- Evaluation loss (326.1104) and evaluation metrics MAE (16.7734) are much higher compared to the metrics in the training. This means that the model is not generalising well to the new unseen data, which indicates the possibility of overfitting.

Since there is a high chance that this ANN model created is overfitting the training data, the model should have had better hyperparameter tuning, or the model could have been less complex like reducing number of layers or neurons.
